# Landsat Collection and ...
#### Description of content here:

### Intro Section

In [ ]:
import os
import ee
import folium

In [ ]:
ee.Initialize()

In [ ]:
import geemap

In [ ]:
start = ee.Date.fromYMD(2021,1,1)
end = ee.Date.fromYMD(2022,1,1)

In [ ]:
iDate = '2021-01-01'
fDate = '2022-01-01'

In [5]:
nw_2box = ee.FeatureCollection('projects/ee-pierdubo/assets/nw_2box')

In [6]:
gmw16z = ee.FeatureCollection("projects/ee-pierdubo/assets/gmw_2016_diss_zones")
zone = 20
pxlval = 23
gmw_sel = (gmw16z
           .filter(ee.Filter.And(
               ee.Filter.eq('pxlval', pxlval),
               ee.Filter.gte('area_ha', 0)
           )))

In [7]:
nw1 = (nw_2box
       .filter(ee.Filter.eq('sel', 1)))

In [8]:
# AOI for NWestern area, using a polyline
aoi = ee.Geometry.LineString(
    [[49.09178474992574, -11.924273142945154],
     [48.66331795305074, -13.275179628809104],
     [47.818428215464124, -14.41247011042455],
     [48.179964869881715, -14.846057452723358],
     [48.08998151060017, -15.275048458710872],
     [47.753046040851736, -14.483073303752759],
     [46.60887459367574, -15.668473695347696],
     [44.40062264055074, -16.196686141581498]])

In [ ]:
geom = ee.Geometry(nw1)

In [ ]:
geom = aoi

In [9]:
geom = ee.Geometry.Rectangle([[48.4684, -12.6752], [48.9465, -13.6273]])

### Add Earth Engine dataset

In [ ]:
PALSAR_2020 = ee.Image("JAXA/ALOS/PALSAR/YEARLY/SAR/2020")

In [ ]:
jers_nw_mos = ee.Image("projects/ee-pierdubo/assets/Jers_madag_NW_mos")

In [ ]:
srtm = ee.Image("USGS/SRTMGL1_003")
srtm1 = ee.Image("NASA/NASADEM_HGT/001")
jrcImage = ee.Image("JRC/GSW1_2/GlobalSurfaceWater")
lc_esa = ee.Image("ESA/GLOBCOVER_L4_200901_200912_V2_3").select('landcover')

In [ ]:
gmw1996 = ee.FeatureCollection("projects/ee-pierdubo/assets/GMW_001_GlobalMangroveWatch_1996")
Mangr = ee.FeatureCollection("projects/ee-pierdubo/assets/gmw2016_sel1")

### Visualisations Stuff

In [ ]:
# Set visualization parameters for NDVI
ndvi_vis_params = {
    'min': -0.1, 'max': 1,
    'palette': ['04fffe','CE7E45','DF923D','F1B555','FCD163','99B718','74A901','66A000',
                '529400','3E8601','207401','056201','004C00','023B01','012E01','011D01','011301']    
}

In [ ]:
# Set visualization parameters for MVI
mvi_vis_params = {
    'min': -1, 'max': 10,
    'palette': ['04fffe','CE7E45','DF923D','F1B555','FCD163','99B718','74A901','66A000',
                '529400','3E8601','207401','056201','004C00','023B01','012E01','011D01','011301']    
}

In [ ]:
# Set visualization parameters for land cover.
lcEsa_vis_params = {
    'min':11, 'max': 230,
    'palette': ['aaefef','ffff63','dcef63','cdcd64','006300','009f00','aac700','003b00','286300',
                '788300','8d9f00','bd9500','956300','ffb431','ffebae','00785a','009578','00dc83',
                'c31300','fff5d6','0046c7','ffffff','743411']
}

In [ ]:
# Set visualization parameters for ground elevation.
elv_vis_params = {
    'min': 0, 'max': 4000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']
}

In [ ]:
# Set visualization parameters for anything with a lot of classes
idx_vis_params = {
    'min':-1, 'max': 1,
    'palette':["ffffff","e2e097","dfe22f","fff705","ffd611","ffb613","ff8b13","ff6e08","ff500d","ff0000","de0101",
               "c21301","0602ff","235cb1","307ef3","269db1","30c8e2","32d3ef","3be285","3ff38f","86e26f"]
}

### Cloud Masking Various Stuff

In [12]:
def maskL8sr(image):
    cloudShadowBitMask = 1 << 3    
    cloudsBitMask = 1 << 5 
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
    .And(qa.bitwiseAnd(cloudsBitMask).eq(0))  
    return image.updateMask(mask).divide(10000) \
    .select("B[0-9]*") \
    .copyProperties(image, ["system:time_start"])

In [ ]:
# Alternate vers. for cloud masking Landsat 8 imagery (source: ccdc Parevalo)
def maskClouds(image):
    quality = image.select('BQA')
    cloud01 = quality.eq(61440)
    cloud02 = quality.eq(53248)
    cloud03 = quality.eq(28672)
    mask = cloud01.Or(cloud02).Or(cloud03).Not()
    return image.updateMask(mask)

In [ ]:
# Function to cloud mask from the QA60 (Sentinel-2) and from the pixel_qa band of Landsat 8 SR data
def maskS2clouds(image):
    QA60 = image.select(['QA60'])
    clouds = QA60.bitwiseAnd(1 << 10).Or(QA60.bitwiseAnd(1 << 11))
    return image.updateMask(clouds.Not())

### Prepare Landsat 4 to 8, and maybe 9

In [13]:
# Prepare Landsat 8 with strict filtering of noisy pixels
# @param {ee.Image} image, (e.g.) Landsat SR image with pixel_qa band
# @returns {ee.Image} Landsat image with masked noisy pixels
#
def prepareL8(image):
    bandList = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10']
    nameList = ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2', 'TEMP']
    scaling = [10000, 10000, 10000, 10000, 10000, 10000, 1000]
    validTOA = [66, 68, 72, 80, 96, 100, 130, 132, 136, 144, 160, 164]
    validQA = [322, 386, 324, 388, 836, 900]
    scaled = ee.Image(image).select(bandList).rename(nameList).divide(ee.Image.constant(scaling))
    mask1 = ee.Image(image).select(['pixel_qa']).remap(validQA, ee.List.repeat(1, len(validQA)), 0) # validQA.length
    mask2 = image.select('radsat_qa').eq(0)
    mask3 = image.select(bandList).reduce(ee.Reducer.min()).gt(0)
    mask4 = ee.Image(image).select(['sr_aerosol']).remap(validTOA, ee.List.repeat(1, len(validTOA)), 0) # validTOA.length
    return ee.Image(image).addBands(scaled).updateMask(mask1.And(mask2).And(mask3).And(mask4))

# Prepare Landsat 7 with strict filtering of noisy pixels
# @param {ee.Image} image Landsat SR image with pixel_qa band
# @returns {ee.Image} Landsat image with masked noisy pixels
#
def prepareL7(image):
    bandList = ['B1', 'B2','B3','B4','B5','B7','B6']
    nameList = ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2', 'TEMP']
    scaling = [10000, 10000, 10000, 10000, 10000, 10000, 1000]
    scaled = ee.Image(image).select(bandList).rename(nameList).divide(ee.Image.constant(scaling))
    validQA = [66, 130, 68, 132]
    mask1 = ee.Image(image).select(['pixel_qa']).remap(validQA, ee.List.repeat(1, len(validQA)), 0) # validQA.length
    # Gat valid data mask, for pixels without band saturation
    mask2 = image.select('radsat_qa').eq(0)
    mask3 = image.select(bandList).reduce(ee.Reducer.min()).gt(0)
    # Mask hazy pixels. Aggressively filters too many images in arid regions (e.g Egypt)
    # unless we force include 'nodata' values by unmasking
    mask4 = image.select("sr_atmos_opacity").unmask().lt(300)
    # Slightly erode bands to get rid of artifacts due to scan lines
    mask5 = ee.Image(image).mask().reduce(ee.Reducer.min()).focal_min(2.5)
    return ee.Image(image).addBands(scaled).updateMask(mask1.And(mask2).And(mask3).And(mask4).And(mask5))

# Prepare Landsat 4 and 5 with strict filtering of noisy pixels
# @param {ee.Image} image Landsat SR image with pixel_qa band
# @returns {ee.Image} Landsat image with masked noisy pixels
#
def prepareL4L5(image):
    bandList = ['B1', 'B2','B3','B4','B5','B7','B6']
    nameList = ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2', 'TEMP']
    scaling = [10000, 10000, 10000, 10000, 10000, 10000, 1000]
    scaled = ee.Image(image).select(bandList).rename(nameList).divide(ee.Image.constant(scaling))
    validQA = [66, 130, 68, 132]
    mask1 = ee.Image(image).select(['pixel_qa']).remap(validQA, ee.List.repeat(1, len(validQA)), 0) #
  # Gat valid data mask, for pixels without band saturation
    mask2 = image.select('radsat_qa').eq(0)
    mask3 = image.select(bandList).reduce(ee.Reducer.min()).gt(0)
  # Mask hazy pixels. Aggressively filters too many images in arid regions (e.g Egypt)
  # unless we force include 'nodata' values by unmasking
    mask4 = image.select("sr_atmos_opacity").unmask().lt(300)
    return ee.Image(image).addBands(scaled).updateMask(mask1.And(mask2).And(mask3).And(mask4))

# Prepare Collection 2 Landsat 4, 5, and 7 with strict filtering of noisy pixels
# @param {ee.Image} image Landsat SR image with pixel_qa band
# @returns {ee.Image} Landsat image with masked noisy pixels
#
def prepareL4L5L7Col2(image):
    bandList = ['SR_B1','SR_B2','SR_B3','SR_B4','SR_B5','SR_B7','ST_B6']
    nameList = ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2', 'TEMP']
    subBand = ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2']
    
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2);
    thermalBand = image.select('ST_B6').multiply(0.00341802).add(149.0);
    scaled = opticalBands.addBands(thermalBand, None, True).select(bandList).rename(nameList);
    validQA = [5440, 5504]  # 5442，5506
    mask1 = ee.Image(image).select(['QA_PIXEL']).remap(validQA, ee.List.repeat(1, len(validQA)), 0)
    # Gat valid data mask, for pixels without band saturation
    mask2 = image.select('QA_RADSAT').eq(0)
    mask3 = scaled.select(subBand).reduce(ee.Reducer.min()).gt(0)
    mask4 = scaled.select(subBand).reduce(ee.Reducer.max()).lt(1)
    # Mask hazy pixels using AOD threshold
    mask5 = (image.select("SR_ATMOS_OPACITY").unmask(-1)).lt(300)
    return ee.Image(image).addBands(scaled).updateMask(mask1.And(mask2).And(mask3).And(mask4).And(mask5))

# Prepare Collection 2 Landsat 8 with strict filtering of noisy pixels
# @param {ee.Image} image Landsat SR image with pixel_qa band
# @param {Boolean} switch between with/without mask
# @returns {ee.Image} Landsat image with masked noisy pixels
#
def prepareL8Col2(image):
    bandList = ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'ST_B10']
    nameList = ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2', 'TEMP']
    subBand = ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2']
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBand = image.select('ST_B10').multiply(0.00341802).add(149.0)
    scaled = opticalBands.addBands(thermalBand, None, True).select(bandList).rename(nameList)
    validTOA = [2, 4, 32, 66, 68, 96, 100, 130, 132, 160, 164]
    validQA = [21824, 21888] # 21826, 21890
    mask1 = ee.Image(image).select(['QA_PIXEL']).remap(validQA, ee.List.repeat(1, len(validQA)), 0)
    mask2 = image.select('QA_RADSAT').eq(0)
  # Assume that all saturated pixels equal to 20000
    mask3 = scaled.select(subBand).reduce(ee.Reducer.min()).gt(0)
    mask4 = scaled.select(subBand).reduce(ee.Reducer.max()).lt(1)
    mask5 = ee.Image(image).select(['SR_QA_AEROSOL']).remap(validTOA, ee.List.repeat(1, len(validTOA)), 0)
    return ee.Image(image).addBands(scaled).updateMask(mask1.And(mask2).And(mask3).And(mask4).And(mask5))    

### Calculate / Add Indices individually (same as in 'indices1.ipynb')

Images need to be 'prepared' first, which consists of renaming bands to standardardized versions

In [ ]:
# 1. Calculate/addBands NDVI for an image
# @param {ee.Image} image,  e.g., Landsat image with NIR and RED bands
# @returns {ee.Image} NDVI image
#
def calcNDVI(image):
    ndvi = ee.Image(image).normalizedDifference(['NIR','RED']).rename('NDVI')
    return ndvi

def addNDVI(image):
    ndvi = ee.Image(image).normalizedDifference(['NIR','RED']).float().rename('NDVI')
    return image.addBands(ndvi)

In [ ]:
# 2. Calculate 'Green' NDVI for an image
# @param {ee.Image} image,  e.g., Landsat image with NIR and RED bands
# @returns {ee.Image} GNDVI image
#
def addGNDVI(image):
    gndvi = ee.Image(image).normalizedDifference(['NIR','GREEN']).float()
    return image.addBands(gndvi.rename('gNDVI'))

In [ ]:
# 3. Calculate Standard Deviation of NDVI values over an image collection
#
def addNDVIsd(image):
    ndviStd = ee.ImageCollection(image).select('NDVI').reduce(ee.Reducer.stdDev())
    # reducer = ee.Reducer.stdDev()
    # ndviStd = ndvi.reduce(ee.Reducer.median())
    # return image.addBands(ndviStd.rename('ndviSD'))
    return ndviStd.rename('ndviSD')

In [ ]:
# 'addNDVIsd' needs some finessing (??); here, we run the 'reducer' directly on the LsColl:
ndviSD = lsColl.select('NDVI').reduce(ee.Reducer.stdDev())

In [ ]:
# 4. Calculate NBR for an image
# @param {ee.Image} image  Landsat image with NIR and SWIR2 bands
# @returns {ee.Image} NBR image
#
def calcNBR(image):
    nbr = ee.Image(image).normalizedDifference(['NIR', 'SWIR2']).rename('NBR')
    return nbr

In [ ]:
# 5. Calculate EVI for an image
# @param {ee.Image} image Landsat image with NIR, RED, and BLUE bands
# @returns {ee.Image} EVI transform
#
def calcEVI(image):
    evi = ee.Image(image).expression(
          'float(2.5*(((B4) - (B3)) / ((B4) + (6 * (B3)) - (7.5 * (B1)) + 1)))',
          {
              'B4': ee.Image(image).select(['NIR']),
              'B3': ee.Image(image).select(['RED']),
              'B1': ee.Image(image).select(['BLUE'])
          }).rename('EVI')
    return evi

In [ ]:
# 6. Calculate EVI2
#
def calcEVI2(image):
    evi2 = ee.Image(image).expression(
        'float(2.5*(((B4) - (B3)) / ((B4) + (2.4 * (B3)) + 1)))',
        {
            'B4': image.select('NIR'),
            'B3': image.select('RED')
        }).rename('EVI2')
    return evi2

In [ ]:
# 7. Calculate/addBands: NDMI
#
def addNDMI(image):
    ndmi = ee.Image(image).normalizedDifference(['NIR', 'SWIR1']).float().rename('NDMI')
    return image.addBands(ndmi)

In [ ]:
# 8. Calculate/addBands: NDWI
#
def addNDWI(image):
    ndwi = ee.Image(image).normalizedDifference(['GREEN', 'NIR']).float().rename('NDWI')
    return image.addBands(ndwi)

In [ ]:
# 9. Calculate and addBands: MNDWI
#
def calcMNDWI(image):
    mndwi = ee.Image(image).normalizedDifference(['GREEN','SWIR1']).float().rename('MNDWI')
    return mndwi

def addMNDWI(image):
    mndwi = ee.Image(image).normalizedDifference(['GREEN','SWIR1']).float().rename('MNDWI')
    return image.addBands(mndwi)

In [ ]:
# 10. Calc/addBands: MVI (Mangrove Vegetation Index, see SEPAL)
#
def addMVI(image):
    mvi = ee.Image(image).expression(
        '(1.0 * (NIR - GREEN) / abs(SWIR1 - GREEN))',
        {
            'GREEN': image.select('GREEN'),
            'NIR': image.select('NIR'),
            'SWIR1': image.select('SWIR1')
        }).float().rename('MVI')
    return image.addBands(mvi)

In [ ]:
# 11. Calculate/addBands: ARI2 [Sentinel-2 only]
#
def addARI2(image):
    ari2 = ee.Image(image).expression(
        '((1.0 / GREEN) - (1.0 / Redg1)) * Redg3',
        {
            'GREEN': image.select(['GREEN']),
            'Redg1': image.select(['Redg1']),  # 'B5'
            'Redg3': image.select(['Redg3'])   # 'B7'
        }).float().rename('ARI2')
    return image.addBands(ari2.rename('ARI2'))

In [ ]:
# 12. Calculate/addBands: ChlR [Sentinel-2 only]
#
def addchlR(image):
    chlRedg = ee.Image(image).expression(
        'pow((Redg3 / Redg1), -1.0)',
        {
            'Redg1': image.select(['Redg1']),
            'Redg3': image.select(['Redg3'])
        }).float().rename('ChlR')
    return image.addBands(chlRedg)

In [ ]:
# 13. Calculate/addBands: SAVI
#
def addSAVI(image):
    savi = ee.Image(image).expression(
        '(NIR - RED)/ (NIR + RED + 0.428) * (1.0 + 0.428)',
        {
            'NIR': image.select('NIR'),
            'RED': image.select('RED')
        }).float().rename('SAVI')
    return image.addBands(savi)

In [ ]:
# 14. Calculate/addBands: BSI (bare soil index)
#
def addBSI(image):
    bsi = ee.Image(image).expression(
        '((SWIR1 + RED) - (NIR + BLUE)) / ((SWIR1 + RED) + (NIR + BLUE))',
        {
            'BLUE': image.select('BLUE'),
            'RED': image.select('RED'),
            'NIR': image.select('NIR'),
            'SWIR1': image.select('SWIR1')
        }).float().rename('BSI')
    return image.addBands(bsi)

In [ ]:
# 15. Calculate/addBands: IBI (index-based built-up index)
# 
def addIBI(image):
    ibiA = ee.Image(image).expression('2 * SWIR1 / (SWIR1 + NIR)',
                                      {
                                          'SWIR1': image.select('SWIR1'), # S2: 'B11'
                                          'NIR': image.select('NIR'), # S2: 'B8'
                                      }).rename('IBI_A')
    ibiB = ee.Image(image).expression('(NIR / (NIR + RED)) + (GREEN / (GREEN + SWIR1))',
                                      {
                                          'NIR': image.select('NIR'),
                                          'RED': image.select('RED'),
                                          'GREEN': image.select('GREEN'),
                                          'SWIR1': image.select('SWIR1')
                                      }).rename('IBI_B')
    ibiAB = ibiA.addBands(ibiB)
    ibi = ibiAB.normalizedDifference(['IBI_A', 'IBI_B'])
    return image.addBands(ibi.rename('IBI'))

In [ ]:
 # 16. Calculate NDFI using endmembers from Souza et al., 2005
 # @param {ee.Image} Surface reflectance image with 6 bands (i.e. not thermal)
 # @returns {ee.Image} NDFI transform
 #
def calcNDFI(image):
  # Do spectral unmixing #
    gv = [.0500, .0900, .0400, .6100, .3000, .1000]
    shade = [0, 0, 0, 0, 0, 0]
    npv = [.1400, .1700, .2200, .3000, .5500, .3000]
    soil = [.2000, .3000, .3400, .5800, .6000, .5800]
    cloud = [.9000, .9600, .8000, .7800, .7200, .6500]
    cf = .1 # Not parameterized
    cfThreshold = ee.Image.constant(cf)
    unmixImage = (ee.Image(image).unmix([gv, shade, npv, soil, cloud], True,True)
                  .rename(['band_0', 'band_1', 'band_2','band_3','band_4']))
    newImage = ee.Image(image).addBands(unmixImage)
    mask = unmixImage.select('band_4').lt(cfThreshold) # Check that this is the right 'image'; previously 'Image'
    ndfi = ee.Image(unmixImage).expression(
        '((GV / (1 - SHADE)) - (NPV + SOIL)) / ((GV / (1 - SHADE)) + NPV + SOIL)', 
        {
            'GV': ee.Image(unmixImage).select('band_0'),
            'SHADE': ee.Image(unmixImage).select('band_1'),
            'NPV': ee.Image(unmixImage).select('band_2'),
            'SOIL': ee.Image(unmixImage).select('band_3')
    })
    return ee.Image(newImage) \
        .addBands(ee.Image(ndfi).rename(['NDFI'])) \
        .select(['band_0','band_1','band_2','band_3','NDFI']) \
        .rename(['GV','Shade','NPV','Soil','NDFI']) \
        .updateMask(mask)

In [ ]:
# 17. Tassel Cap coefficients from Crist 1985
# @param {ee.Image} image, Landsat image with BLUE, GREEN, RED, NIR, SWIR1, and SWIR2
# @returns {ee.Image} 3-band image with Brightness, Greenness, and Wetness
#
def tcTrans(image):
    # Calculate tasseled cap transformation
    brightness = image.expression(
        '(L1 * B1) + (L2 * B2) + (L3 * B3) + (L4 * B4) + (L5 * B5) + (L6 * B6)',
        {
            'L1': image.select('BLUE'),
            'B1': 0.2043,
            'L2': image.select('GREEN'),
            'B2': 0.4158,
            'L3': image.select('RED'),
            'B3': 0.5524,
            'L4': image.select('NIR'),
            'B4': 0.5741,
            'L5': image.select('SWIR1'),
            'B5': 0.3124,
            'L6': image.select('SWIR2'),
            'B6': 0.2303
        })
    greenness = image.expression(
        '(L1 * B1) + (L2 * B2) + (L3 * B3) + (L4 * B4) + (L5 * B5) + (L6 * B6)',
        {
            'L1': image.select('BLUE'),
            'B1': -0.1603,
            'L2': image.select('GREEN'),
            'B2': -0.2819,
            'L3': image.select('RED'),
            'B3': -0.4934,
            'L4': image.select('NIR'),
            'B4': 0.7940,
            'L5': image.select('SWIR1'),
            'B5': -0.0002,
            'L6': image.select('SWIR2'),
            'B6': -0.1446
        })
    wetness = image.expression(
        '(L1 * B1) + (L2 * B2) + (L3 * B3) + (L4 * B4) + (L5 * B5) + (L6 * B6)',
        {
            'L1': image.select('BLUE'),
            'B1': 0.0315,
            'L2': image.select('GREEN'),
            'B2': 0.2021,
            'L3': image.select('RED'),
            'B3': 0.3102,
            'L4': image.select('NIR'),
            'B4': 0.1594,
            'L5': image.select('SWIR1'),
            'B5': -0.6806,
            'L6': image.select('SWIR2'),
            'B6': -0.6109
        })

    bright =  ee.Image(brightness).rename('BRIGHTNESS')
    green = ee.Image(greenness).rename('GREENNESS')
    wet = ee.Image(wetness).rename('WETNESS')

    tasseledCap = ee.Image([bright, green, wet])
    return tasseledCap

### Do Indices

In [ ]:
from ipynb.fs.full.indices1 import *

In [ ]:
# Calculate spectral indices for all bands in collection
# @param {ee.ImageCollection} collection Landsat image collection
# @returns {ee.ImageCollection} Landsat image with spectral indices
#
def doIndices(image):
    def func_gev(image):
        NDVI = calcNDVI(image)
        gNDVI = addGNDVI(image)
        NBR = calcNBR(image)
        EVI = calcEVI(image)
        EVI2 = calcEVI2(image)
        NDMI = addNDMI(image)
        NDWI = addNDWI(image)
        MNDWI = addMNDWI(image)
        MVI = addMVI(image)
        BSI = addBSI(image)
        IBI = addIBI(image)
        SAVI = addSAVI(image)
        TC = tcTrans(image)
      # NDFI function requires surface reflectance bands only
        BANDS = ['BLUE','GREEN','RED','NIR','SWIR1','SWIR2']
        NDFI = calcNDFI(image.select(BANDS))
        return image.addBands([NDVI, gNDVI, NBR, EVI, EVI2, NDMI, NDWI, MNDWI, MVI, BSI, IBI, SAVI, TC, NDFI])
    return image.map(func_gev)

In [ ]:
targetBands = ['BLUE','GREEN','RED','NIR','SWIR1','SWIR2','NDVI','gNDVI','NBR','EVI','NDMI','NDWI','MNDWI',
               'MVI','BSI','IBI','SAVI','BRIGHTNESS','GREENNESS','WETNESS']

In [ ]:
image = lsColl.median()

In [ ]:
# doesn't work ?? What if I add '.median()' hmmm!!
#
ndviSD = lsColl.map(addNDVIsd)

In [ ]:
ndviSD = addNDVIsd(lsColl).select('ndviSD')

In [ ]:
## This method for running 'fun' over my Landsat Collection (lsColl) works *** remember to calculate median ***
ndvi2 = lsColl.map(calcNDVI).median()

In [ ]:
image = image.addBands(ndvi2)

In [ ]:
# xx. Direct outputs from JRC's GSW1_3 Global Surface Water:
#
jrcImage = ee.Image("JRC/GSW1_3/GlobalSurfaceWater")

In [ ]:
# Add the 6 standard JRC Bands to the image stack, or (see next):
#
image = image.addBands(jrcImage.unmask(0))

In [ ]:
# xx. Retrieve JRC / addBands and rename (only affects 'seasonality' —> 'seasonalit', 10-char max.):  
#
def addJRC(image):
    # Update the mask (??)
    image = image.unmask(0)
    image = image.addBands(jrcImage.select(['occurrence']).rename(['occurrence']))
    image = image.addBands(jrcImage.select(['change_abs']).rename(['change_abs']))
    image = image.addBands(jrcImage.select(['change_norm']).rename(['change_nor']))
    image = image.addBands(jrcImage.select(['seasonality']).rename(['seasonalit']))
    image = image.addBands(jrcImage.select(['transition']).rename(['transition']))
    image = image.addBands(jrcImage.select(['max_extent']).rename(['max_extent']))
    return image

In [ ]:
image = image.addBands(addJRC(jrcImage))

In [ ]:
# Three DEM versions, NASADEM should be an improvement ... need to check the band contents, as it differs between versions ***
#
srtm = ee.Image("USGS/SRTMGL1_003")
srtm1 = ee.Image("NASA/NASADEM_HGT/001")
aDem = ee.Image("JAXA/ALOS/AW3D30/V3_2").select('DSM')

In [ ]:
import math

In [ ]:
# xx. Calculate/addBands Topographic Indices: (i) slope and aspect; (ii) east- and northness
#
import math
def addTopo(image):
    topo = ee.Algorithms.Terrain(image)
  # From aspect (a), calculate 'eastness (sin a), northness (cos a)
    deg2rad = ee.Number(math.pi).divide(180)
    aspect = topo.select('aspect')
    aspect_rad = aspect.multiply(deg2rad)
    eastness = aspect_rad.sin().rename('eastness').float()
    northness = aspect_rad.cos().rename('northness').float()
    
    topo = topo.select(['slope', 'aspect']).addBands(eastness).addBands(northness)
    image = image.addBands(topo)
    return image

In [ ]:
image = image.addBands(addTopo(srtm))

In [ ]:
# Testing outpus from 'ee.Algorithms.Terrain':
#
topo = ee.Algorithms.Terrain(srtm)

### Collect Landsat image by sensor

In [ ]:
collection8 = (ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
    .filterDate(iDate, fDate)
    .map(prepareL8))

In [ ]:
collection7 = (ee.ImageCollection('LANDSAT/LE07/C01/T1_SR')
    .filterDate(iDate, fDate)
    .map(prepareL7))

In [ ]:
collection5 = (ee.ImageCollection("LANDSAT/LT05/C01/T1_SR")
    .filterDate(iDate, fDate)
    .map(prepareL4L5))

In [ ]:
collection4 = (ee.ImageCollection("LANDSAT/LT04/C01/T1_SR")
    .filterDate(iDate, fDate)
    .map(prepareL4L5))

### Get Landsat Collection, various methods, (i) generateCollection; (ii) collMerge; (iii) getLandsat

In [ ]:
def getLandsat(iDate, fDate, geom, targetBands, collection):
    # collection = (options)
    # iDate = (options) 
    # fDate = (options)
    # region = (options)
    # targetBands = (options)
    
    if (collection == 1):
        collection8 = (ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') 
            .filterDate(iDate, fDate))
        collection7 = (ee.ImageCollection('LANDSAT/LE07/C01/T1_SR') 
            .filterDate(iDate, fDate))
        collection5 = (ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') 
            .filterDate(iDate, fDate))
        collection4 = (ee.ImageCollection('LANDSAT/LT04/C01/T1_SR') 
            .filterDate(iDate, fDate))
        coll8 = collection8.map(prepareL8)
        coll7 = collection7.map(prepareL7)
        coll5 = collection5.map(prepareL4L5)
        coll4 = collection4.map(prepareL4L5)
        
    elif (collection == 2):
        collection8 = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') 
            .filterDate(iDate, fDate))
        collection7 = (ee.ImageCollection('LANDSAT/LE07/C02/T1_L2') 
            .filterDate(iDate, fDate))
        collection5 = (ee.ImageCollection('LANDSAT/LT05/C02/T1_L2') 
            .filterDate(iDate, fDate))
        collection4 = (ee.ImageCollection('LANDSAT/LT04/C02/T1_L2') 
            .filterDate(iDate, fDate))
        coll8 = collection8.map(prepareL8Col2)
        coll7 = collection7.map(prepareL4L5L7Col2)
        coll5 = collection5.map(prepareL4L5L7Col2)
        coll4 = collection4.map(prepareL4L5L7Col2)
        
    coll = coll4.merge(coll5).merge(coll7).merge(coll8)
    if (geom):
        coll = coll.filterBounds(geom)
    indices = doIndices(coll).select(targetBands)
    
  ## If we want to exclude some of the sensor: but not sure how to, e.g., 'no sensors.l4' 
    # if (not sensors.l4):
    #     indices = indices.filterMetadata('SATELLITE','not_equals','LANDSAT_4')
    # if (not sensors.l5):
    #     indices = indices.filterMetadata('SATELLITE','not_equals','LANDSAT_5')        
    # if (not sensors.l7):
    #     indices = indices.filterMetadata('SATELLITE','not_equals','LANDSAT_7')        
    # if (not sensors.l8):
    #     indices = indices.filterMetadata('SATELLITE','not_equals','LANDSAT_8')        
        
    return ee.ImageCollection(indices)

In [ ]:
lsColl = getLandsat(iDate, fDate, aoi, targetBands, 2)

In [ ]:
iDate, fDate, lsColl

In [ ]:
geom = nw1

In [ ]:
sensors = {'l4', 'l5', 'l7', 'l8'}

In [ ]:
aoi

In [ ]:
getLandsatParams = {
    'collection': 1,
    'start': iDate, 
    'end': fDate, 
    # 'startDoy': 1, 'endDoy': 366,
    'region': geom,
    'targetBands': ['BLUE','GREEN','RED','NIR','SWIR1','SWIR2', 'NDVI'],
    'useMask': True,
    'sensors': {'l4': True, 'l5': True, 'l7': True, 'l8': True},
}
getLandsatParams

In [19]:
Map.addLayer(lsColl_noind.select(['SWIR1','NIR','GREEN']).median(), {'bands': ['SWIR1','NIR','GREEN'], 'min':0.020, 'max':0.400, 'gamma': 0.85}, 'Landsat Coll')

In [ ]:
Map.addLayer(lsColl.select(['MVI']).median(), mvi_vis_params, 'Index Sel')

In [18]:
Map.addLayer(lsColl_noind, {}, 'Landsat all bands')

In [23]:
Map.addLayer(ndvi, {}, 'Image in progress')

In [ ]:
Map.addLayer(aDem, elv_vis_params, 'ALOS DEM')

In [22]:
Map = geemap.Map(center=(-13.5, 48.4), zoom=8)
Map

Map(center=[-13.5, 48.4], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children…

In [10]:
# Get Landsat image collection (T1_SR), no indices
#
def generateCollection(geom, iDate, fDate, collection):
    collection = 1
    collection8 = (ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
        .filter("WRS_ROW < 122")
        .filterBounds(geom)
        .map(prepareL8))
    collection7 = (ee.ImageCollection('LANDSAT/LE07/C01/T1_SR')
        .filter("WRS_ROW < 122")
        .filterBounds(geom)
        .map(prepareL7))
    collection5 = (ee.ImageCollection('LANDSAT/LT05/C01/T1_SR')
        .filter("WRS_ROW < 122")
        .filterBounds(geom)
        .map(prepareL4L5))
    collection4 = (ee.ImageCollection('LANDSAT/LT04/C01/T1_SR')
        .filter("WRS_ROW < 122")
        .filterBounds(geom)    
        .map(prepareL4L5))
    
    mergedColl = ee.ImageCollection(collection8).merge(collection7).merge(collection5).merge(collection4).filterDate(iDate, fDate)
    return mergedColl
    # return ee.ImageCollection(mergedColl)

In [14]:
lsColl_noind = generateCollection(geom, iDate, fDate, 1)

In [16]:
iDate, fDate, lsColl_noind

('2021-01-01',
 '2022-01-01',
 <ee.imagecollection.ImageCollection at 0x1d522ee1550>)

In [20]:
from ipynb.fs.full.indices1 import calcNDVI

In [21]:
ndvi = lsColl_noind.map(calcNDVI).median()

In [ ]:
# Get Landsat image collection, earlier version targetting 'T1_SR' only, and calc/addBands indices
#
def collMerge(iDate, fDate, region):
    collection8 = (ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
        .filterDate(iDate, fDate)
        .map(prepareL8))
    collection7 = (ee.ImageCollection('LANDSAT/LE07/C01/T1_SR')
        .filterDate(iDate, fDate)
        .map(prepareL7))
    collection5 = (ee.ImageCollection('LANDSAT/LT05/C01/T1_SR')
        .filterDate(iDate, fDate)
        .map(prepareL4L5))
    collection4 = (ee.ImageCollection('LANDSAT/LT04/C01/T1_SR')
        .filterDate(iDate, fDate)
        .map(prepareL4L5))
    col = collection4.merge(collection5).merge(collection7).merge(collection8)
    if (region):
        col = col.filterBounds(region)
    indices = doIndices(col).select(targetBands)
    return ee.ImageCollection(indices)

In [ ]:
lsColl_t1SR = collMerge(iDate, fDate, geom)

### Next thing:

In [ ]:
# tc = collection8.map(tcTrans)

In [ ]:
# indices = doIndices(collection8)